In [0]:
import os
import random

import numpy as np
import pandas as pd

import matplotlib
import matplotlib.pyplot as plt
matplotlib.rcParams['figure.figsize'] = [5, 5]
matplotlib.rcParams['figure.dpi'] = 200

import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
from torchvision import transforms
import torchvision.models as models


from data_helper import UnlabeledDataset, LabeledDataset
#from my_data_helper import UnlabeledDataset, LabeledDataset
from helper import collate_fn, draw_box

from collections import OrderedDict


from resnet import resnet18

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
if torch.cuda.is_available():
  device = torch.device("cuda")
else:
  device = torch.device("cpu")

In [0]:
# image_folder = '../data'
# annotation_csv = '../data/annotation.csv'

image_folder = '/content/drive/My Drive/student_data/data'
annotation_csv = '/content/drive/My Drive/student_data/data/annotation.csv'

random.seed(0)
np.random.seed(0)
torch.manual_seed(0);

# You shouldn't change the unlabeled_scene_index
# The first 106 scenes are unlabeled
unlabeled_scene_index = np.arange(106)
# The scenes from 106 - 133 are labeled
# You should devide the labeled_scene_index into two subsets (training and validation)
labeled_scene_index = np.arange(106, 134)


train_index = np.arange(106,124)
val_index = np.arange(124,134)
test_index = np.arange(132,134)

In [7]:
print(device)

cuda


In [0]:
# implementation from https://github.com/kuangliu/torchcv/blob/master/torchcv/utils/box.py
# with slight modifications
def box_iou(boxes1, boxes2):
    """
    Return intersection-over-union (Jaccard index) of boxes.
    Both sets of boxes are expected to be in (x1, y1, x2, y2) format.
    Arguments:
        boxes1 (Tensor[N, 4])
        boxes2 (Tensor[M, 4])
    Returns:
        iou (Tensor[N, M]): the NxM matrix containing the pairwise
            IoU values for every element in boxes1 and boxes2
    """
    area1 = box_area(boxes1)
    area2 = box_area(boxes2)

    lt = torch.max(boxes1[:, None, :2], boxes2[:, :2])  # [N,M,2]
    rb = torch.min(boxes1[:, None, 2:], boxes2[:, 2:])  # [N,M,2]

    wh = (rb - lt).clamp(min=0)  # [N,M,2]
    inter = wh[:, :, 0] * wh[:, :, 1]  # [N,M]

    iou = inter / (area1[:, None] + area2 - inter).type(torch.double)
    return iou


def box_area(boxes):
    """
    Computes the area of a set of bounding boxes, which are specified by its
    (x1, y1, x2, y2) coordinates.
    Arguments:
        boxes (Tensor[N, 4]): boxes for which the area will be computed. They
            are expected to be in (x1, y1, x2, y2) format
    Returns:
        area (Tensor[N]): area for each box
    """
    return (boxes[:, 2] - boxes[:, 0]) * (boxes[:, 3] - boxes[:, 1])


def get_offsets(gt_boxes, ex_boxes):

    ex_width = ex_boxes[:, 2] - ex_boxes[:, 0]
    ex_height = ex_boxes[:, 3] - ex_boxes[:, 1]
    ex_center_x = ex_boxes[:, 0] + 0.5*ex_width
    ex_center_y = ex_boxes[:, 1] + 0.5*ex_height

    gt_width = gt_boxes[:, 2] - gt_boxes[:, 0]
    gt_height = gt_boxes[:, 3] - gt_boxes[:, 1]
    gt_center_x = gt_boxes[:, 0] + 0.5*gt_width
    gt_center_y = gt_boxes[:, 1] + 0.5*gt_height


    delta_x = (gt_center_x - ex_center_x) / ex_width
    delta_y = (gt_center_y - ex_center_y) / ex_height
    delta_scaleX = torch.log(gt_width / ex_width)
    delta_scaleY = torch.log(gt_height / ex_height)

    offsets = torch.cat([delta_x.unsqueeze(0), 
                    delta_y.unsqueeze(0),
                    delta_scaleX.unsqueeze(0),
                    delta_scaleY.unsqueeze(0)],
                dim=0)
    return offsets.permute(1,0)

def get_bbox_gt(bboxes1, classes, gt_boxes, sz):
  
  bboxes = bboxes1.clone()
  bboxes *= 10
  bboxes = bboxes + 400
  classes += 1
  high_threshold = 0.7
  low_threshold = 0.3
  ex1 = bboxes[:, 0, 3].unsqueeze(0)
  ey1 = bboxes[:, 1, 3].unsqueeze(0)
  ex2 = bboxes[:, 0, 0].unsqueeze(0)
  ey2 = bboxes[:, 1, 0].unsqueeze(0)
  ex_boxes = torch.cat([ex1, ey1, ex2, ey2], dim=0)
  ex_boxes = ex_boxes.permute(1,0)

  ex_width = ex_boxes[:, 2] - ex_boxes[:, 0]
  ex_height = ex_boxes[:, 3] - ex_boxes[:, 1]
  ex_center_x = ex_boxes[:, 0] + 0.5*ex_width
  ex_center_y = ex_boxes[:, 1] + 0.5*ex_height

  gt_widths = gt_boxes[:, 2] - gt_boxes[:, 0]
  gt_heights = gt_boxes[:, 3] - gt_boxes[:, 1]
  gt_center_x = gt_boxes[:, 0] + 0.5*gt_widths
  gt_center_y = gt_boxes[:, 1] + 0.5*gt_heights

  ious = box_iou(gt_boxes, ex_boxes)
  # ious = ious.permute(1,0)
  vals, inds = torch.max(ious, dim=1)
  gt_classes = torch.zeros((sz*sz*4)).type(torch.long)
  gt_offsets = torch.zeros((sz*sz*4, 4)).type(torch.double)

  # HEATMAP CODE
  # gt_scores = gt_classes.clone()
  # gt_scores[vals > 0.001] = 1
  # gt_heat_map_x = gt_center_x[gt_scores == 1]
  # gt_heat_map_y = gt_center_y[gt_scores == 1]

  # plotMap(gt_heat_map_x, gt_heat_map_y)

  gt_classes[vals > high_threshold] = classes[inds[vals > high_threshold]] # foreground anchors
  gt_classes[vals < low_threshold] = 0 # background anchors
  gt_classes[(vals >= low_threshold) & (vals < high_threshold)] = -1 # anchors to ignore

  actual_boxes = ex_boxes[inds[vals > high_threshold]]
  ref_boxes = gt_boxes[vals > high_threshold]
  g_offsets = get_offsets(ref_boxes, actual_boxes)
  gt_offsets[vals > high_threshold] = g_offsets

  return gt_classes, gt_offsets



def get_gt_boxes():
    '''
    Return a matrix with size 4 x 2560000
    2560000 = 800*800*4 -> since target BEV map of size 800*800, with 4 different scales
    '''
    scaleX = [100, 70, 50, 20]
    scaleY = [25, 20, 15, 5]
    map_sz = 800
    widths = torch.tensor(scaleX)
    heights = torch.tensor(scaleY)
    ref_boxes = []
    for x in range(map_sz):
        for y in range(map_sz):
            x_r = widths + x
            y_r = heights + y
            x_l = torch.tensor([x, x, x, x])
            y_l = torch.tensor([y, y, y, y])
            x_r = x_r.unsqueeze(0)
            y_r = y_r.unsqueeze(0)
            x_l = x_l.unsqueeze(0)
            y_l = y_l.unsqueeze(0)
            ref_box = torch.cat((x_l, y_l, x_r, y_r))
            ref_box = ref_box.permute((1,0))
            ref_boxes.append(ref_box)

    gt_boxes = torch.stack(ref_boxes).view(-1,4).type(torch.double)
    return gt_boxes


def my_collate_fn(batch):
    images = []
    gt_boxes = get_gt_boxes()
    img_h = 256
    img_w = 306
    map_sz = 800
    class_target = []
    box_target = []
    for x in batch:
        
        # stack images, torch.Size([BatchSize, 6*3, 256, 306])
        images.append(x[0])
        
        gt_classes, gt_offsets = get_bbox_gt(x[1]['bounding_box'], x[1]['category'], gt_boxes, map_sz)
        class_target.append(gt_classes)
        box_target.append(gt_offsets)

    samples = torch.stack(images)
    samples = samples.view(len(batch), -1, img_h, img_w).double()
    class_target = torch.stack(class_target)
    box_target = torch.stack(box_target)
    return samples, class_target, box_target

In [0]:
class BoundingBox(nn.Module):

    def __init__(self):
        super().__init__()
        self.encoder = resnet18()
        self.classifier = nn.Conv2d(512, 10, kernel_size=3, padding=1, bias=False)
        self.input_shape = (800,800)

        self.regressor = nn.Conv2d(10, 4*4, kernel_size=3, padding=1, bias=False)
        self.pred = nn.Conv2d(10, 4*9, kernel_size=3, padding=1, bias=False)
    
    def forward(self, x):
        x = self.encoder(x)
        x = self.classifier(x)
        x = F.interpolate(x, size=self.input_shape, mode='bilinear', align_corners=False)

        pred_x = self.pred(x)
        box_x = self.regressor(x)
        return pred_x, box_x

In [0]:
def compute_loss(out_pred, class_targets, val=False):
  '''
  This function is to calculate category prediction loss
  '''
  # Sample equal number of positive and negative anchors
  out_pred = out_pred.permute((0,2,1))

  if val == True:
      good_targets = class_targets[class_targets != -1]
      good_preds = out_pred[class_targets != -1]
      if good_preds.shape[0] == 0:
          return torch.tensor(0)
      return F.cross_entropy(good_preds, good_targets)

  bad_examples = class_targets[class_targets == -1]
  foreground_examples = class_targets[class_targets > 0]
  foreground_preds = out_pred[class_targets > 0]
  background_examples = class_targets[class_targets == 0]
  background_preds = out_pred[class_targets == 0]

  num_pos = foreground_examples.shape[0]

  if num_pos == 0:
      print('No positive anchors found in this image !!')
      return torch.tensor(0)

  perm1 = torch.randperm(background_examples.shape[0], device=device)[:]
  background_examples = background_examples[perm1]
  background_preds = background_preds[perm1]

  targets = torch.cat((background_examples, foreground_examples), dim=0)
  preds = torch.cat((background_preds, foreground_preds), dim=0)

  #loss = focal_loss(preds, targets)
  #return loss
  return F.cross_entropy(preds, targets)



def bbox_loss(box_targets, class_targets, out_bbox):
  '''
  this function is to calculate bbox loss
  '''
  inds = (class_targets != 0)
  box_targets = box_targets[inds]
  out_bbox = out_bbox[inds]
  loss_bbox = F.smooth_l1_loss(out_bbox.double(), box_targets.double())
  return loss_bbox        

In [0]:
epochs = 10
batch_sz = 2

best_val_loss = 100
model = BoundingBox().to(device)
param_list = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.Adam(param_list, lr=1e-4)


transform = torchvision.transforms.ToTensor()

labeled_trainset = LabeledDataset(image_folder=image_folder,
                                  annotation_file=annotation_csv,
                                  scene_index=train_index,
                                  transform=transform,
                                  extra_info=True
                                 )
trainloader = torch.utils.data.DataLoader(labeled_trainset, batch_size=2, shuffle=False, num_workers=2, collate_fn=my_collate_fn)

labeled_valset = LabeledDataset(image_folder=image_folder,
                                  annotation_file=annotation_csv,
                                  scene_index=val_index,
                                  transform=transform,
                                  extra_info=True
                                 )
valloader = torch.utils.data.DataLoader(labeled_valset, batch_size=2, shuffle=False, num_workers=2, collate_fn=my_collate_fn)

In [0]:

for epoch in range(epochs):
    model.train()
    train_losses = []
    
    for i, (samples, class_target, box_target) in enumerate(trainloader):
        
        #samples = samples
        
        out_pred, out_bbox = model(samples.float().to(device))
        out_bbox = out_bbox.view(batch_sz, -1, 4)
        out_pred = out_pred.view(batch_sz, 9, -1)

        ## EDIT: calculating bbox loss 
        #loss = compute_loss(out_pred, class_target.to(device))
        loss = bbox_loss(box_target.to(device), class_target.to(device), out_bbox)
        train_losses.append(loss.item())

        if loss.item() != 0:
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            #self.scheduler.step()
        
        #train_losses.append(loss.item())

        #loss.backward()
        optimizer.step()
        if i % 10 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, i * len(samples), len(trainloader.dataset),
                10. * i / len(trainloader), loss.item()))
            
    print("\nAverage Train Epoch Loss: ", np.mean(train_losses))
    
    
    model.eval()
    val_losses = []
    
    for i, (samples, class_target, box_target) in enumerate(valloader):
        #model.eval()
        
        #class_target = class_target.to(device)
        #box_target = box_target.to(device)
        out_pred, out_bbox = model(samples.to(device))
        out_bbox = out_bbox.view(batch_sz, -1, 4)
        out_pred = out_pred.view(batch_sz, 9, -1)
        
        ## EDIT: calculating bbox loss 
        #loss = compute_loss(out_pred, class_targets.to(device), val=True)
        loss = bbox_loss(box_target.to(device), class_target.to(device), out_bbox)
        
        val_losses.append(loss.item())
        
        if i % 10 == 0:
            print('Val Epoch: {} [{}/{} ({:.0f}%)]\tAverage Loss So Far: {:.6f}'.format(
                epoch, i * len(samples), len(valloader.dataset),
                5. * i / len(valloader), np.mean(val_losses)))
            
    print("Average Validation Epoch Loss: ", np.mean(val_losses))
#     global best_val_loss
#     if np.mean(val_losses) < best_val_loss:
#         best_val_loss = np.mean(val_losses)
#         torch.save(model.state_dict(), 'best_val_loss_counting_simple.pt')

Train Epoch: 0 [0/2268 (0%)]	Loss: 0.082864
